# Tugas Besar 1 IF3270 Pembelajaran Mesin <br /> Feedforward Neural Network

## Kelompok 39

- Dzaky Satrio Nugroho - 13522059
- Julian Caleb Simandjuntak - 13522099
- Rafiki Prawhira Harianto - 13522065

In [1]:
# Import dulu
import numpy as np


In [ ]:
# Fungsi Aktivasi 

class ActivationFunction:
    
    # Fungsi linear
    @staticmethod
    def linear(x: np.ndarray) -> np.ndarray:
        return x

    # Fungsi ReLU
    @staticmethod
    def relu(x: np.ndarray) -> np.ndarray:
        return np.maximum(0, x)

    # Fungsi Sigmoid
    @staticmethod
    def sigmoid(x: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-x))

    # Fungsi Hyperbolic Tangent
    @staticmethod
    def tanh(x: np.ndarray) -> np.ndarray:
        return np.tanh(x)

    # Fungsi Softmax
    @staticmethod
    def softmax(x: np.ndarray) -> np.ndarray:
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0)

    # Fungsi Leaky ReLU
    @staticmethod
    def leaky_relu(x: np.ndarray, alpha=0.1) -> np.ndarray:
        return np.maximum(alpha*x, x)

    # Fungsi Swish
    @staticmethod
    def swish(x: np.ndarray) -> np.ndarray:
        return x * sigmoid(x)

In [3]:
# Fungsi Loss

class LossFunction:
    
    # Mean Squared Error
    @staticmethod
    def mse(y_pred: np.ndarray, y_true: np.ndarray) -> float:
        mse = np.sum((y_pred - y_true) ** 2) / len(y_true)
        return mse

    # Binary Cross-Entropy
    @staticmethod
    def bce(y_pred: np.ndarray, y_true: np.ndarray) -> float:
        bce = -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)).mean()
        return bce

    # Categorical Cross-Entropy
    @staticmethod
    def cce(y_pred: np.ndarray, y_true: np.ndarray) -> float:
        cce = -1 / len(y_true) * np.sum(np.sum(y_true * np.log(y_pred)))
        return cce

In [ ]:
import numpy as np

"""
Inisialisasi 1 layer bobot dengan parameter wajib shape yang merupakan tuple berisi ukuran matrix bobot
Contoh: 
shape=(3, 4) berarti:
- Untuk layer dengan 3 neuron awal dan layer dengan 4 neuron berikutnya
- Menghasilkan matrix bobot dengan 3 baris dan 4(+1 bias) kolom
"""
class WeightInitializer:
    @staticmethod
    def zeros(shape, bias=1):
        w = np.zeros(shape)
        b = np.ones((shape[0], bias))
        return np.hstack((b, w))

    @staticmethod
    def uniform(shape, bias=1, lower_bound=-0.1, upper_bound=0.1, seed=None):
        if seed is not None:
            np.random.seed(seed)
        w = np.random.uniform(lower_bound, upper_bound, shape)
        b = np.ones((shape[0], bias))
        return np.hstack((b, w))

    @staticmethod
    def normal(shape, bias=1, mean=0.0, variance=1.0, seed=None):
        if seed is not None:
            np.random.seed(seed)
        std_dev = np.sqrt(variance)  # Konversi variance ke standard deviation
        w = np.random.normal(mean, std_dev, shape)
        b = np.ones((shape[0], bias))
        return np.hstack((b, w))
    
# Contoh penggunaan
# zero_weights = WeightInitializer.zeros(shape)
# print(zero_weights)
# output:
#  [[1. 0. 0. 0. 0.]
#  [1. 0. 0. 0. 0.]
#  [1. 0. 0. 0. 0.]